# Introduction

In this notebook we check whether the [repeating FRB](http://adsabs.harvard.edu/cgi-bin/bib_query?arXiv:1603.00581) events are distributed according to [Poisson statistics](https://en.wikipedia.org/wiki/Poisson_distribution), using the [Anderson Darling test](https://en.wikipedia.org/wiki/Anderson%E2%80%93Darling_test).

# Mathematical Formulation

Suppose we observe a system for a time $T$, and detect $N$ events, at times $t_i$. According to Poisson statistics, the average number of events increases linearly with time, as is the standard deviation

$C_t \left( t \right) = \sigma^2 \left(t \right) = \lambda t$

where $\lambda$ is a constant. We want to compare this theoretical prediction to the experimental event count

$C_e \left(t\right) = \sum_{i=1}^{N} \theta \left(t - t_i \right)$

Where $\theta \left(t\right)$ is the [Heaviside step function](https://en.wikipedia.org/wiki/Heaviside_step_function). The Anderson Darling fit quality is

$\chi^2 = \frac{1}{T}\int_0^T \frac{\left(C_t\left(t \right ) - C_e \left(t \right )\right )^2}{\sigma^2 \left(t\right)}dt = \frac{1}{2} \lambda T - 2N + 2\sum_{i=1}^N \frac{t_i}{T} + \frac{1}{\lambda T} \sum_{i=1}^N \left(2 i - 1 \right ) \ln \frac{T}{t_i} $

The best fit rate $\lambda$ can be found by minimising $\chi^2$

$\lambda = \frac{1}{T}\sqrt{2 \sum_{i=1}^N \left(2i-1\right) \ln \frac{T}{t_i}}$

Substituting back into the expression for $\chi^2$ yields the minimum value of the fit quality

$\chi^2_{\min} = \sqrt{2 \sum_{i=1}^N \left(2i-1 \right ) \ln \frac{T}{t_i}} - 2N+2 \sum_{i=1}^N \frac{t_i}{T}$

We note that due to the fact that the Poisson distribution describes a memory - less process, the observation window does not have to be contiguous.

# Application for Repeating FRB

Load the raw observational data

In [4]:
import numpy

times_txt = numpy.loadtxt('code/times.txt', skiprows=1)
observations_txt = numpy.loadtxt('code/observations.txt', skiprows=1)

Total cumulative observation time

In [19]:
total_cumulative_observation_time = numpy.sum(observations_txt.T[-2])

Calculate correction for disperion measure

In [8]:
import astropy.time
dm_correction = [astropy.time.TimeDelta(4.148808e3*559/line[7]**2, format='sec') for line in observations_txt]

Geographical correction for light travel time

In [10]:
import astropy.coordinates
frb_loc = astropy.coordinates.SkyCoord('05:31:58','+33:08:04',
                                       unit=(astropy.units.hourangle,astropy.units.deg),equinox='J2000')
arecibo_loc = astropy.coordinates.EarthLocation.from_geodetic(lon=-66.7528,lat=18.3464) #index 0
effelsberg_loc = astropy.coordinates.EarthLocation.from_geodetic(lon='6:52:58',lat='50:31:29') #index 1
gbt_loc = astropy.coordinates.EarthLocation.from_geodetic(lon='-79.8398',lat='38.4322') #index 2
vla_loc = astropy.coordinates.EarthLocation.from_geodetic(lon='-107.6184',lat='34.0784') #index 3
lovell_loc = astropy.coordinates.EarthLocation.from_geodetic(lon='-2.3085',lat='53.2367') #index 4
locs = [arecibo_loc,effelsberg_loc,gbt_loc,vla_loc,lovell_loc] #locations of all involved observatories
light_travel_times = [astropy.time.Time('%04i-%02i-%02iT%02i:%02i:%02i'%(line[0],
                                                    line[1],
                                                    line[2],
                                                    line[3],
                                                    line[4],
                                                    line[5]),
                  format='isot',
                   scale='utc',
                  location=locs[int(line[6])]).light_travel_time(frb_loc)
             for line in observations_txt]

/home/ayalin/anaconda3/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


Calculate observation start time in mjd and apply the corrections above

In [12]:
start_times_raw = [astropy.time.Time('%04i-%02i-%02iT%02i:%02i:%02i'%(line[0],
                                                                line[1],
                                                                line[2],
                                                                line[3],
                                                                line[4],
                                                                line[5]),
                              format='isot',
                               scale='utc',
                              location=locs[int(line[6])])
             for line in observations_txt]

start_mjd = [(tstart.tdb-dmcorr+tlt).mjd 
             for tstart,dmcorr,tlt in 
             zip(start_times_raw, dm_correction, light_travel_times)]



Calculate the cumulative times for the FRB events in seconds (this is the time measured from the begininng of the first observation if all observation windows are concatenated)

In [13]:
t_frb_cumulative = numpy.zeros_like(times_txt.T[0])
for n, line in enumerate(times_txt):
    idx = int(line[-1])
    idx_in_observations = numpy.where(observations_txt.T[-1]>0)[0][idx]
    t_frb_cumulative[n] = numpy.sum(observations_txt.T[-2][:idx_in_observations+1]) + (line[0]-start_mjd[idx_in_observations])*3600

Calculate $\chi^2_{\min}$

In [23]:
def anderson_darling_poisson_test(t2T_list):
    
    n_list = numpy.linspace(1,len(t2T_list), len(t2T_list))
    return numpy.sqrt(-2*numpy.sum((2*n_list-1)*numpy.log(t2T_list)) - 2*len(t2T_list) + 2*numpy.sum(t2T_list))

numpy.sqrt(anderson_darling_poisson_test(t_frb_cumulative/ total_cumulative_observation_time))

5.805395958327793

The distance between the theory and the observations is almost six standard deviations, so Poisson statistics can be disqualified with a high degree of certainty.